# Maquina Virtual

Este codigo implementa uma máquina virtual (interpretador) com cache para memória principal com as seguintes funcionalidades relacionadas a um set de instruções criado:
 - buscar
 - decodificar
 - executar

Definimos o tamanho da arquitetura para 16 bits<br>
O set de instruções criado possui dois tipos de instrução:<br>
### Tipo 1: operações aritméticas
`[tipo(4 bits)] | [end. reg 1(4 bits)] | [end. reg 2(4 bits)] | [end. reg Dest(4 bits)]`


### Tipo 2: operações de **Load**, **Store** e **Set**
`[tipo(4 bits)] | [end. reg(4 bits)] | [end. mem(8 bits)]`



In [1]:
architecture = 16

Cada codigo de instrução terá 4 bits 

In [2]:
op_size = 4

Definimos 10 registradores e 256 espaços de memória do programa e inicializamos todos com 0

In [3]:
register_count = 10
registers = [0] * register_count

program_memory_count = 256
program_memory = [0] * program_memory_count

Inicializa o program counter

In [4]:
pc = 0

Definimos as instruções e seus valores:

In [5]:
instr_to_code = {'add'   : 1, # 0x01
                 'sub'   : 2, # 0x02
                 'store' : 3, # 0x03
                 'load'  : 4, # 0x04
                 'set'   : 5, # 0x05
                 'exit'  : 0} # 0x00

<br><br>
## Funções ajudantes
<br>Para ajudar, criamos um dicionário que faz o mapeamento contrário das instruções:

In [6]:
code_to_instr = {v : k for k, v in instr_to_code.items()}

Importamos uma biblioteca de manipulação de bits

In [7]:
from bitarray import bitarray

No bloco abaixo, criamos um decorador para nossas funções normais

In [8]:
from functools import wraps
def next_instruction_decorator(func):
    @wraps(func)
    def decorator(*args, **kwargs):
        func(*args, **kwargs)
        return pc + 1

    return decorator

Sua função é retornar sempre o valor do PC + 1 para funções que não possuem pulos (implementação futura)

Criamos então algumas funções para ajudar durante a execução do programa:

Uma para converter bytes para int

In [9]:
def bytes_to_int(bs):
    result = 0

    for b in bs:
        result = result * 256 + int(b)

    return result

Uma para converter um vetor de bits para int

In [10]:
def bitarray_to_int(ba):
    return bytes_to_int(ba.tobytes())

Uma para preencher um vetor de bits incompleto para ficar múltiplo de 8 e pronto para ser convertido para bytes

In [11]:
def fill_bitarray(ba):
    ba.reverse()
    ba.fill()
    ba.reverse()
    return ba

Duas para quebrar os dados decodificados de acordo com o instruction set:<br>
Para tipo 1

In [12]:
def break_instr_arithmetic(instr_data):
    return map(bitarray_to_int, map(fill_bitarray, [instr_data[:4], instr_data[4:8], instr_data[8:12]]))

E para tipo 2

In [13]:
def break_instr_memory_manip(instr_data):
    return map(bitarray_to_int, map(fill_bitarray, [instr_data[:4], instr_data[4:12]]))

Pronto, agora temos o basico para implementar nossa máquina virtual<br>
<br>
## Implementação das operações
<br>
A primeira instrução a ser implementada será ADD, que salva a soma dos registradores fonte no registrador de destino

In [14]:
@next_instruction_decorator
def instr_add(instr_data):
    source_a, source_b, dest = break_instr_arithmetic(instr_data)
    registers[dest] = registers[source_a] + registers[source_b]
    print(f'\tR{dest} = R{source_a} + R{source_b}')

Em seguida o SUB, que salva o resultado do registrador A - o registrador B no registrador de destino

In [15]:
@next_instruction_decorator
def instr_sub(instr_data):
    source_a, source_b, dest = break_instr_arithmetic(instr_data)
    registers[dest] = registers[source_a] - registers[source_b]
    print(f'\tR{dest} = R{source_a} - R{source_b}')

Então o LOAD, que carrega um valor da memória para um registrador

In [16]:
@next_instruction_decorator
def instr_load(instr_data):
    dest_reg, source_mem = break_instr_memory_manip(instr_data)
    print(f'\tR{dest_reg} = M{source_mem}')
    registers[dest_reg] = cache.fetch_data(source_mem)

STORE, que salva o valor de um registrador na memória

In [17]:
@next_instruction_decorator
def instr_store(instr_data):
    source_reg, dest_mem = break_instr_memory_manip(instr_data)
    print(f'\tM{dest_mem} = R{source_reg}')
    cache.set_data(dest_mem, registers[source_reg])

SET, que guarda um valor imediato no registrador de destino

In [18]:
@next_instruction_decorator
def instr_set(instr_data):
    source_reg, raw_value = break_instr_memory_manip(instr_data)
    registers[source_reg] = raw_value
    print(f'\tR{source_reg} = {raw_value}')

E EXIT, que fecha o programa enviando um código de saída definido pelo programa

In [19]:
@next_instruction_decorator
def instr_exit(instr_data):
    _, raw_value = break_instr_memory_manip(instr_data)
    print('\tThe End')
    exit(raw_value)

Agora que já estão definidas as funções da máquina virtual, podemos mapear as operações aos nomes.<br>
Usaremos um dicionário para o mapeamento:

In [20]:
instruction_map = {'add'   : instr_add,
                   'sub'   : instr_sub,
                   'store' : instr_store,
                   'load'  : instr_load,
                   'set'   : instr_set,
                   'exit'  : instr_exit}

<br><br>
## Decodificação
Função para decodificar as instruções

In [21]:
def decode(instruction):
    print('\n\nDecoding', ''.join([ instruction[i*4 : i*4 + 4].to01() for i in range(len(instruction)//4) ]) )
    
    instr = instruction[:op_size]
    fill_bitarray(instr)
    instr = bitarray_to_int(instr)
    instr = code_to_instr[instr]
    print('instruction: ', instr)
    
    instr_data = instruction[op_size:]
    print('data: ', instr_data.to01())
    
    return instruction_map.get(instr, exit)(instr_data)

<br><br>
## Geração de arquivo para testes
Aqui definimos uma função para "compilar" um programa teste para um arquivo

In [22]:
def generate_example_file(file_p):
    ex_instructions = [
        ['set',   [0, 6]],     # r0 = 6
        ['set',   [1, 9]],     # r1 = 9
        ['add',   [0, 1, 2]],  # r2 = r0 + r1
        ['add',   [2, 2, 2]],  # r2 *= 2
        ['sub',   [2, 1, 3]],  # r3 = r2 - r1
        ['store', [3, 16]],     # m8 = r3       --> miss -> update -> set -> now dirty
        ['store', [3, 0]],     # m0 = r3        --> miss -> dirty -> flush -> update -> set -> now dirty
        ['load',  [4, 0]],     # r4 = m0        --> hit
        ['load',  [5, 16]],     # r5 = m8       --> miss -> dirty -> flush -> update 
        ['store', [0, 16]],     # m16 = r0      --> hit -> set -> now dirty
        ['add',   [4, 1, 0]]]  # r0 = r4 + r1
    
    ex_instructions_s = []
    print('Arquivo gerado:')
    for count, i in enumerate(ex_instructions):
        instr = bin(instr_to_code[i[0]])[2:].rjust(op_size, '0')
        if len(i[1]) == 2:
            instr += bin(i[1][0])[2:].rjust(4, '0')
            instr += bin(i[1][1])[2:].rjust(8, '0')
            
        elif len(i[1]) == 3:
            instr += bin(i[1][0])[2:].rjust(4, '0')
            instr += bin(i[1][1])[2:].rjust(4, '0')
            instr += bin(i[1][2])[2:].rjust(4, '0')
            
        ex_instructions_s.append(instr)
        print(count, '-', instr)
    
    convert_to_bytes = lambda s: int(s, 2).to_bytes(len(s) // 8, byteorder='big')
    with open(file_p, 'wb') as fh:
        for i in ex_instructions_s:
            fh.write(convert_to_bytes(i))

<br><br>
## Leitura do arquivo
Entrada do nome do arquivo:

In [23]:
file_path = None
if file_path is None:
    file_path = input('File: ')
    if file_path == '':
        file_path = 'exemplo.bin'

File: exemplo.bin


Se o arquivo não existe, geraremos nosso exemplo

In [24]:
from os import path
if not path.isfile(file_path):
    generate_example_file(file_path)

Arquivo gerado:
0 - 0101000000000110
1 - 0101000100001001
2 - 0001000000010010
3 - 0001001000100010
4 - 0010001000010011
5 - 0011001100010000
6 - 0011001100000000
7 - 0100010000000000
8 - 0100010100010000
9 - 0011000000010000
10 - 0001010000010000


O arquivo então é lido

In [25]:
def read_instructions(file_path):
    with open(file_path, 'rb') as f:
        return f.read()

E separado em instruções

In [26]:
program_bytes = read_instructions(file_path)
instructions = bitarray(endian='big')
instructions.frombytes(program_bytes)
instruction_count = len(instructions) // architecture
instructions = [instructions[i * (architecture) : (i + 1) * (architecture)] for i in range(instruction_count)]
print('instructions:', len(instructions))

instructions: 11


<h1>cache</h1>

Definimos como é uma linha de nossa cache, com tag, validação, indicador de mudanças e words

In [27]:
class CacheLine:
    def __init__(self, words_per_line):
        self.words_per_line = words_per_line
        self.is_valid = False
        self.tag = 0
        self.is_dirty = True
        self.words = [0] * words_per_line

E criamos a cache, que possui linhas e atributos ajudantes

In [28]:
class Cache:
    def __init__(self, words_per_line, line_count):
        self.words_per_line = words_per_line
        self.line_count = line_count
        self.lines = [CacheLine(words_per_line) for i in range(line_count)]
        
        import math
        self.offset_bit_count = int(math.log(words_per_line, 2))
        self.line_bit_count = int(math.log(line_count, 2))

Adicionamos uma função para copiar os dados modificados da cache para a memória principal

In [29]:
def _CacheLine_flush(self, base):
    for i, word in enumerate(self.words):
        program_memory[base + i] = word
        
    self.is_dirty = False
    print('Cache line flushed')
    
CacheLine.flush = _CacheLine_flush

Funções para atualizar a cache com a memória principal copiando da segunda para a primeira

In [30]:
def _CacheLine_update(self, tag, base):
    for i in range(self.words_per_line):
        self.words[i] = program_memory[base + i]
        
    self.is_valid = True
    self.is_dirty = False
    self.tag = tag
    print('Cache line updated')
    
    
CacheLine.update = _CacheLine_update
        
        
def _Cache_update_line(self, tag, line_number):
    base = ((tag << self.offset_bit_count) + line_number) << self.line_bit_count
    self.lines[line_number].update(tag, base)
        
        
Cache.update_line = _Cache_update_line

E então as funções principais, uma para buscar dados da cache

In [31]:
def _Cache_fetch_data(self, address):
    word_number = address & self.offset_bit_count
    line_number = (address >> self.offset_bit_count) & (2**self.line_bit_count - 1)
    desired_tag = address >> (self.line_bit_count + self.offset_bit_count)
    print('tag:', desired_tag, 'line:', line_number, 'word:', word_number)
    target_line = self.lines[line_number]
    
    fetch_from_memory = False
    if target_line.is_valid:
        # may be a hit..
        if target_line.tag != desired_tag:
            # miss :(
            fetch_from_memory = True
            if target_line.is_dirty:
                # flush before repopulate
                base = ((target_line.tag << self.offset_bit_count) + line_number) << self.line_bit_count
                target_line.flush(base)
    
    else:
        # miss, repopulate
        fetch_from_memory = True
        
    if fetch_from_memory:
        self.update_line(desired_tag, line_number)
    else:
        print('hit')
        
        
    return self.lines[line_number].words[word_number]
    
    
Cache.fetch_data = _Cache_fetch_data

E uma para setar dados da cache

In [32]:
def _Cache_set_data(self, address, value):
    word_number = address & self.offset_bit_count
    line_number = (address >> self.offset_bit_count) & (2**self.line_bit_count - 1)
    desired_tag = address >> (self.line_bit_count + self.offset_bit_count)
    print('tag:', desired_tag, 'line:', line_number, 'word:', word_number)
    target_line = self.lines[line_number]
    
    fetch_from_memory = False
    if target_line.is_valid:
        if target_line.is_dirty:
            base = ((target_line.tag << self.offset_bit_count) + line_number) << self.line_bit_count
            target_line.flush(base)
            
        if target_line.tag != desired_tag:
            fetch_from_memory = True
    
    else:
        # miss, repopulate
        fetch_from_memory = True
        
    if fetch_from_memory:
        self.update_line(desired_tag, line_number)
        
    print('value set')
    target_line.words[word_number] = value
    target_line.is_dirty = True
        
Cache.set_data = _Cache_set_data

E finalmente criamos nossa cache, com 4 palavras por linha e 4 linhas, totalizando 32 bytes

Tag:<br/>
`[tag (12 bits)] | [linha (2 bits)] | [palavra (2 bits)]`

In [33]:
cache = Cache(words_per_line = 4, line_count = 4)

<br><br>
## Programa rodando
Aqui que a mágica começa, finalmente. As instruções então são executadas uma a uma até que seja encontrado um pulo (não implementado nesta versão)

In [34]:
while pc < instruction_count:
    pc = decode(instructions[pc])



Decoding 0101000000000110
instruction:  set
data:  000000000110
	R0 = 6


Decoding 0101000100001001
instruction:  set
data:  000100001001
	R1 = 9


Decoding 0001000000010010
instruction:  add
data:  000000010010
	R2 = R0 + R1


Decoding 0001001000100010
instruction:  add
data:  001000100010
	R2 = R2 + R2


Decoding 0010001000010011
instruction:  sub
data:  001000010011
	R3 = R2 - R1


Decoding 0011001100010000
instruction:  store
data:  001100010000
	M16 = R3
tag: 1 line: 0 word: 0
Cache line updated
value set


Decoding 0011001100000000
instruction:  store
data:  001100000000
	M0 = R3
tag: 0 line: 0 word: 0
Cache line flushed
Cache line updated
value set


Decoding 0100010000000000
instruction:  load
data:  010000000000
	R4 = M0
tag: 0 line: 0 word: 0
hit


Decoding 0100010100010000
instruction:  load
data:  010100010000
	R5 = M16
tag: 1 line: 0 word: 0
Cache line flushed
Cache line updated


Decoding 0011000000010000
instruction:  store
data:  000000010000
	M16 = R0
tag: 1 line: 0 

<br/><br/>
E após o término da execução, podemos sair

In [35]:
print('End of execution')
exit(0)

End of execution
